In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
import random
from sklearn.model_selection import train_test_split
path = "/content/drive/MyDrive/TFM/"
path1 = path + "Planets.csv"
path2 = path + "Not_Planets.csv"

In [3]:
#Creación del dataset con los registros de secuencias positivas y secuencias negativas
planets_df = pd.read_csv(path1, names=list(range(4000)))
not_planets_df = pd.read_csv(path2, names=list(range(4000)))
X_data = pd.concat([planets_df, not_planets_df]).to_numpy()
y_data = np.concatenate((np.ones(len(planets_df)), np.zeros(len(not_planets_df))))

#Separación en train y test
X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size=0.20, random_state=42)
y_train = np.transpose(np.atleast_2d(y_train))
y_test = np.transpose(np.atleast_2d(y_test))

In [4]:
#Definición del modelo a entrenar
from tensorflow.keras import layers
from tensorflow import keras

model_name = "Conv1D_D"

model = keras.Sequential(
    [
        layers.Input(shape=(4000,1)),
        layers.Conv1D(filters=32, kernel_size=5, activation='relu'),
        layers.MaxPooling1D(pool_size=3),
        layers.Conv1D(filters=64, kernel_size=5, activation='relu'),
        layers.MaxPooling1D(pool_size=3),
        layers.Conv1D(filters=128, kernel_size=5, activation='relu'),
        layers.MaxPooling1D(pool_size=3),
        layers.Dropout(0.2),
        layers.Flatten(),
        layers.Dense(100, activation='sigmoid'),
        layers.Dense(1, activation='sigmoid')
    ],
    name=model_name
)

In [5]:
from datetime import datetime
#Compilación del modelo
model.compile(
    loss=keras.losses.BinaryCrossentropy(),
    optimizer=keras.optimizers.Adam(learning_rate=1e-3),
    metrics=[
              keras.metrics.BinaryAccuracy(),
              keras.metrics.Recall(),
              keras.metrics.Precision(),
              keras.metrics.AUC(num_thresholds=25)
            ]
    )

#Definición del directorio donde guardar los datos del entrenamiento
logdir = "/content/drive/MyDrive/TFM/Logs/" + model.name + "/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir)
model_checkpoint = keras.callbacks.ModelCheckpoint(
    filepath="/content/drive/MyDrive/TFM/Models/" + model_name + "/best_model.keras",
    verbose=1,
    save_best_only=True,
    mode="max",
    monitor="val_binary_accuracy",
)

In [6]:
training_history = model.fit(
    X_train,
    y_train,
    batch_size=64,
    verbose=1, # Suppress chatty output; use Tensorboard instead
    epochs=50,
    validation_data=(X_test, y_test),
    callbacks=[tensorboard_callback, model_checkpoint],
)

Epoch 1/50
520/520 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step - auc: 0.5702 - binary_accuracy: 0.5660 - loss: 0.6682 - precision: 0.5517 - recall: 0.8249
Epoch 1: val_binary_accuracy improved from -inf to 0.62738, saving model to /content/drive/MyDrive/TFM/Models/Conv1D_D/best_model.keras
520/520 ━━━━━━━━━━━━━━━━━━━━ 35s 53ms/step - auc: 0.5702 - binary_accuracy: 0.5660 - loss: 0.6682 - precision: 0.5517 - recall: 0.8250 - val_auc: 0.6097 - val_binary_accuracy: 0.6274 - val_loss: 0.6499 - val_precision: 0.5907 - val_recall: 0.9576
Epoch 2/50
519/520 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step - auc: 0.5768 - binary_accuracy: 0.5803 - loss: 0.6570 - precision: 0.5543 - recall: 0.9488
Epoch 2: val_binary_accuracy did not improve from 0.62738
520/520 ━━━━━━━━━━━━━━━━━━━━ 27s 36ms/step - auc: 0.5768 - binary_accuracy: 0.5803 - loss: 0.6570 - precision: 0.5543 - recall: 0.9487 - val_auc: 0.5636 - val_binary_accuracy: 0.5861 - val_loss: 0.6599 - val_precision: 0.5619 - val_recall: 0.9795
Epoch 3/50
519/520 ━━━